<h3 style="color:green">✅👌 Запись названий снб в пустой дата фрейм </h3>

In [2]:
import pandas as pd
import plotly.express as px
import numpy as np
import pyarrow as pa #Для сохранения DataFrame с мультииндексом в формате Parquet вам потребуется использовать библиотеку pyarrow
import pyarrow.parquet as pq
import pickle
import datetime

In [3]:
predict= pd.read_excel('./forecast/forecast_freg_vol.xlsx') #прогноз
snb_msop = pd.read_excel('./data/snb_msop.xlsx') #справочник мсоп с снб и названиями поставщиков
snb_msop['L-1']= snb_msop['L-1'].str.replace('"', '') # убираю кавычки, не отображаются в таблице с прогнозом

predict.set_index('index',inplace=True) #отдельяем пеовый столбец с датой

<h3 style="color:red"> ЖРС </h3>

In [4]:
grs=snb_msop.loc[snb_msop['L-3'].isin(['Окатыши', 'Аглосырье'])]

predict_columns = predict.columns.values.tolist()
grs_snb_unique = list(grs['SNB'].unique())

common_elements = list(set(predict_columns).intersection(grs_snb_unique))

grs_df = predict.iloc[:, [predict.columns.get_loc(col) for col in common_elements]]
#grs_df.head()

<h3 style="color:red"> Уголь </h3>

In [5]:
coal=snb_msop.loc[snb_msop['L-3'].isin(['Угли технологические'])]

predict_columns = predict.columns.values.tolist()
coal_snb_unique = list(coal['SNB'].unique())

common_elements = list(set(predict_columns).intersection(coal_snb_unique))

coal_df = predict.iloc[:, [predict.columns.get_loc(col) for col in common_elements]]
#coal_df.head()

ЖРС

In [6]:
predict=grs_df

df = pd.DataFrame()

for i in range(len(list(predict))):
    
    df.loc[0, i] = list(snb_msop.loc[snb_msop['SNB'] == list(predict)[i]]['L-1'])
    df.loc[1, i] = list(predict)[i]
#print(df)

a1 = pd.DataFrame(predict.values)
result = pd.concat([ df,a1], axis=0)
#result

In [7]:
date = pd.DataFrame()
date.loc[0,1]='Дата'
date.loc[1,1]=''
date=pd.concat([ date,pd.DataFrame(predict.index )], axis=0).iloc[:,1]
date.iloc[0,]='Дата'
date.iloc[1,]=''
result=pd.concat([ date,result], axis=1) 

In [8]:
total = pd.DataFrame(a1.sum()).T
result = result.append(total, ignore_index=True) #Добавление новой строки в конец DataFrame
result.loc[len(result)-1,'index'] = 'Итого:'

C:\Users\Christina\AppData\Local\Temp\ipykernel_6860\1552358577.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(total, ignore_index=True) #Добавление новой строки в конец DataFrame


In [9]:
#result.to_excel  (r'C:\Users\Komarova149158\Desktop\strimlit\data\result_grs.xlsx')
result.to_excel('./forecast/result_grs.xlsx', index=False)

Уголь

In [10]:
predict=coal_df

df = pd.DataFrame()

for i in range(len(list(predict))):
    
    df.loc[0, i] = list(snb_msop.loc[snb_msop['SNB'] == list(predict)[i]]['L-1'])
    df.loc[1, i] = list(predict)[i]
#print(df)

a1 = pd.DataFrame(predict.values)
result = pd.concat([ df,a1], axis=0)

In [11]:
date = pd.DataFrame()
date.loc[0,1]='Дата'
date.loc[1,1]=''
date=pd.concat([ date,pd.DataFrame(predict.index )], axis=0).iloc[:,1]
date.iloc[0,]='Дата'
date.iloc[1,]=''
result=pd.concat([ date,result], axis=1) 

In [12]:
total = pd.DataFrame(a1.sum()).T
result = result.append(total, ignore_index=True) #Добавление новой строки в конец DataFrame
result.loc[len(result)-1,'index'] = 'Итого:'

C:\Users\Christina\AppData\Local\Temp\ipykernel_6860\1552358577.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append(total, ignore_index=True) #Добавление новой строки в конец DataFrame


In [13]:
#result.to_excel  (r'C:\Users\Komarova149158\Desktop\strimlit\data\result_coal.xlsx')
result.to_excel  ('./forecast/result_coal.xlsx')


In [16]:
snb_msop = pd.read_excel('./data/snb_msop.xlsx')

accuracy= pd.read_excel('./data/accuracy.xlsx')
col = ['SNB','L-1','L-2','Объем по спецификации','Бюджет','Округление до вагонной нормы','date','%_Выполнения_плана']
accuracy = accuracy[col]

predict_grs= pd.read_excel('./data/result_grs.xlsx')
predict_grs.set_index('Unnamed: 0',inplace=True)

predict_coal= pd.read_excel('./data/result_coal.xlsx')
predict_coal.set_index('Unnamed: 0',inplace=True)

predict= pd.read_excel('./data/forecast_freg_vol.xlsx') #прогноз
predict.set_index('index',inplace=True) #отдельяем пеовый столбец с датой

df_all = pd.read_feather('./data/data_2023_strimlit.feather')

In [17]:
start_date = snb_msop['date'].max()
# Convert start_date to Pandas datetime format
start_date = pd.to_datetime(start_date, format='%Y-%d-%m')
start_of_month = start_date - pd.offsets.MonthBegin(1)
end_of_month = start_date + pd.offsets.MonthEnd(0)
# Generate all the dates of the month
dates_of_month = pd.date_range(start=start_date, end=end_of_month, freq='D')

In [29]:
common_elements = 'СНБ|103|00000144'
#common_elements=select_event
fact=df_all.loc[(df_all['СНБ_позиция'] == common_elements) & (df_all['DATAOTGR']>= '2023-11-01')]
fact_two_columns = fact[['DATAOTGR', 'ves_nl']]

if fact_two_columns.shape[0]==0:
   fact_two_columns = pd.DataFrame(columns=['DATAOTGR'])
   new_rows = []
   for date in dates_of_month:
    new_rows.append({'DATAOTGR': date})
    fact_two_columns = pd.concat([fact_two_columns, pd.DataFrame(new_rows)], ignore_index=True)
    fact_two_columns['ves_nl']=0

In [38]:
fact_two_columns

,DATAOTGR,ves_nl
3011,2023-11-01,69.5
3037,2023-11-01,69.5
3130,2023-11-01,68.3
3177,2023-11-01,74.6
3337,2023-11-01,69.6
3374,2023-11-01,68.6
3424,2023-11-01,69.8
3565,2023-11-01,74.6
3824,2023-11-01,69.7
4028,2023-11-01,74.6


In [30]:
fact_two_columns['DATAOTGR'] =fact_two_columns['DATAOTGR'].dt.strftime('%Y-%m-%d')

aggregated_data = fact_two_columns.groupby(['DATAOTGR']).sum()
aggregated_data.reset_index(drop=False , inplace=True)

In [31]:
df = pd.DataFrame(columns=['DATAOTGR'])

# Создаем список новых строк
new_rows = []
for date in dates_of_month:
    new_rows.append({'DATAOTGR': date})

# Конкатенируем новые строки с существующим DataFrame
df = pd.concat([df, pd.DataFrame(new_rows)], ignore_index=True)

df['DATAOTGR'] = df['DATAOTGR'].astype(str)
aggregated_data['DATAOTGR'] = aggregated_data['DATAOTGR'].astype(str)

In [32]:
# Конкатенируем датафреймы по колонкам "DATAOTGR"
result=df.merge(aggregated_data, how='outer')  #cross и т.д
result = result.iloc[:,0:].fillna(0)

predict_columns = predict.columns.values.tolist()

column_index = predict_columns.index(common_elements)
column_index
d1 = predict.iloc[:, column_index]
d1.head()

data = {'Дата': list(d1.index),
        'Прогноз': list(d1.values),
        'Факт':list(result['ves_nl'])
        }

In [33]:
d1.head()

index
2023-11-01       0
2023-11-02    4700
2023-11-03    4700
2023-11-04    4700
2023-11-05    4700
Name: СНБ|103|00000144, dtype: int64

In [34]:
fig1 = px.bar(data, x='Дата', y=['Прогноз','Факт'], barmode='group',color='variable',text='value',
 color_discrete_map={
        'Прогноз': 'rgb(855,195,0)',
        'Факт': 'rgb(0,53,102)'
    }
) #sta text='2'

In [36]:
df_all['DATAOTGR'].max

<bound method NDFrame._add_numeric_operations.<locals>.max of 0       2023-10-15 11:41:55
1       2023-10-06 11:35:52
2       2023-09-27 09:07:43
3       2023-10-29 11:21:24
4       2023-10-29 11:21:24
                ...        
80432   2023-10-20 05:44:10
80433   2023-10-10 05:50:58
80434   2023-10-29 19:26:49
80435   2023-10-31 07:09:42
80436   2023-09-13 05:41:23
Name: DATAOTGR, Length: 80437, dtype: datetime64[ns]>